# XGBoost

In [1]:
import pandas as pd

# Cargar con la librería pandas el dataset bank.csv y mostrarlo
bank_data = pd.read_csv("data/bank.csv", delimiter=";")
bank_data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


In [2]:
# Eliminar la columna "y" del dataset para almacenar el resultado en X (predictores)
# Guardar la columna "y" en la variable y, y convertir los "yes" en 1s y los "no" en 0s.

X = bank_data.drop("y", axis=1)
y = bank_data["y"].map({"yes": 1, "no": 0})

# Mostrar la cardinalidad de X e y.

print(f"Cardinalidad de X: {X.shape}")
print(f"Cardinalidad de y: {y.shape}")

Cardinalidad de X: (4521, 16)
Cardinalidad de y: (4521,)


In [3]:
# Dividir los datos en train y test.
# Tamaño de train: 80%
# Tamaño de test: 20%
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [4]:
# Seleccionar las variables numéricas y las categóricas de los predictores (X_train)

num_features = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_features = X_train.select_dtypes(include=['object']).columns

In [5]:
# Instanciar un transformador StandardScaler para las variables numéricas y un OneHotEncoder para las variables categóricas
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

In [7]:
# Importar la clase ColumnTransformer de scikit-learn
from sklearn.compose import ColumnTransformer

# Usar la clase ColumnTransformer para aplicar el num_transformer
# a las variables numéricas (num_features) y el cat_transformer a las variables categóricas (cat_features)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)

In [11]:
# Crear un pipeline con las fases preprocessor y XGBClassifier
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(eval_metric='logloss'))
])

Hiperparámetros:

**Parámetro `n_estimators`**: Es el número de árboles de decisión que se van a entrenar. Un mayor número deárboles puede incurrir en modelos más complejos.

**Parámetro `learning_rate`**: Un valor de learning rate más bajo hace que se necesiten más árboles para converger hacia una solución más efectiva que puede mejorar la generalización pero requiere más computo. Un valor más alto acelera la convergencia pero con un mayor riesgo de overfitting.

**Parámetro `max_depth`**: Es la profundidad máxima de cada árbol de decisión. Cuanto mayor es el valor de max depth, mayor es la complejidad del modelo y mayor riesgo de overfitting existe.

In [12]:
# Crear una rejilla de hiperparámetros con los siguientes valores posible:
# n_estimators: 100, 200
# learning_rate: 0.01, 0.1
# max_depth: 3, 6

param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__max_depth': [3, 6]
}

In [13]:
%%time

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

CPU times: user 9min 8s, sys: 684 ms, total: 9min 9s
Wall time: 27.1 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object'))])),
                                       ('classifie...
                                                      max_cat_to_onehot=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      random_state=None, ...))]),
             param_grid={'classifier__learning_rate': [0.01, 0.1],
                         'classifier__max_depth': [3, 6],
                         'classifier__n_estimators': [100, 200]},
             scoring='accuracy')

In [14]:
print(f"Mejores hiperparámetros encontrados: {grid_search.best_params_}")

Mejores hiperparámetros encontrados: {'classifier__learning_rate': 0.01, 'classifier__max_depth': 6, 'classifier__n_estimators': 200}


In [18]:
# Obtener las predicciones y el classificacion_report
from sklearn.metrics import classification_report

y_pred = grid_search.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94       807
           1       0.55      0.22      0.32        98

    accuracy                           0.90       905
   macro avg       0.73      0.60      0.63       905
weighted avg       0.87      0.90      0.88       905

